## spliceR (python version) code

#### Import packages

In [1]:
import numpy as np
import pandas as pd
import json
from Bio.Seq import Seq
import math
import urllib.request

#### Helper functions

In [2]:
### reads information from json files from www.ebi.ac.uk HLA database
### example webpage: https://www.ebi.ac.uk/ipd/imgt/hla/alleles/allele/?accession=HLA00050

def parse_json(file_path=None, file_url=None):
    """ takes in path to json file from www.ebi.ac.uk HLA database (default) OR url of json file 
    returns dict of:
    - haplotype_name (str)
    - prevalence (tuple of CWD v2.0 Status, CWD v3.0 Status (Total))
    - genomic_features (list of dicts, one dict for each utr/exon/intron, keys are 'length', 'start', 'type', 'number' (intron/exons only))
    - coding_features (similar to genomic features but only has exons - needed for pct cDNA calculation)
    - sequences (dict of protein, coding, and genomic sequences (probably won't use for BE stuff))
    - genomic_sequence (str)
    """
    d = {}
    if file_url is not None:
        with urllib.request.urlopen(file_url) as url:
            HLA_file_contents = json.load(url)
    else: 
        with open(file_path) as HLA_file:
            HLA_file_contents = json.load(HLA_file)
        # keys we want: 'cwd' (for commonality), 'feature' (exon coords), 'name', 'sequence'
    d['haplotype_name'] = HLA_file_contents['name']
    # d['prevalence'] = (HLA_file_contents['cwd']['version_2'], HLA_file_contents['cwd']['version_3']['Total'])
    d['genomic_features'] = HLA_file_contents['feature']['genomic']
    d['coding_features'] = HLA_file_contents['feature']['coding']
    d['sequences'] = HLA_file_contents['sequence']
    d['genomic_sequence'] = HLA_file_contents['sequence']['genomic']

    return d

In [ ]:
### reads in motif and position weights from tsv files 
def parse_motif_position_weights(motif_weights_filepath, position_weights_filepath):
    """ takes in filepath of tsv files storing motif weights and position weights, 
    returns 2 element tuple of motif_weights_dict, position_weights_dict
        motif_weights_dict is dictionary mapping motif (7 NT str) to dictionary containing weight (float)
        positions_weights_dict is nested dictionary mapping enzyme class (ABE or CBE) to dictionary mapping edited NT position to weight

    example of using weight dictionaries: 
        pw = position_weights_dict["CBE"][target_pos]['position_weight']
        mw = motif_weights_dict[motif]['motif_weight']
    """
    ## motif_weights_dict is dictionary mapping motif (7 NT str) to dictionary containing weight (float)
    ## note that 4th NT must be A or C (for ABE or CBE base editing)
    motif_weights_df = pd.read_csv(motif_weights_filepath, sep="\t", index_col=0)
    motif_weights_dict = motif_weights_df.to_dict('index')

    ## positions_weights_dict is nested dictionary mapping enzyme class (ABE or CBE) to dictionary mapping edited NT position to weight
    position_weights_df = pd.read_csv(position_weights_filepath, sep="\t", index_col=0)
    ABE_pw_df = position_weights_df[position_weights_df['enzyme_class'] == 'ABE']
    ABE_pw_dict = ABE_pw_df.to_dict('index')
    CBE_pw_df = position_weights_df[position_weights_df['enzyme_class'] == 'CBE']
    CBE_pw_dict = CBE_pw_df.to_dict('index')
    position_weights_dict = {"ABE": ABE_pw_dict, "CBE": CBE_pw_dict}

    return motif_weights_dict, position_weights_dict
    

In [4]:
### functions for manipulating DNA seq strings and other basic helper functions

def get_rev_complement(seq):
    comp = str()
    for char in seq:
        if char == "A":
            comp += "T"
        elif char == "T":
            comp += "A"
        elif char == "G":
            comp += "C"
        elif char == "C":
            comp += "G"
    comp = comp[::-1]
    return comp 

def matchDNAPattern(pattern, subject):
    """ takes in a pattern (shorter DNA string) and subject (longer DNA string)
    return list of 2 element lists:
        each list giving start coord of matching region (in longer string) and subsequence of longer string that matches
    note: "N" matches any of the 4 nucleotides ("A", "T", "G", "C")
    """
    output = []
    len_substring = len(pattern)
    len_string = len(subject)
    if len_substring>len_string:
        raise Exception("pattern cannot be longer than subject")
    last_start = len_string-len_substring+1
    for i in range(0,last_start):
        # print(i)
        si = subject[i:i+len_substring]
        m = True
        for idx, char in enumerate(si):
            if (char == pattern[idx]) or (pattern[idx]=="N"):
                continue
            else:
                m = False 
                break
        if m:
            output.append([i,si])
    return(output)

# test
# print(matchDNAPattern("NNGCA", "ACTGCAGGGCA"))
# print(matchDNAPattern("NNGCAT", "ACTGCAGGGCA"))
# print(matchDNAPattern("NNGCAAAAAAAAAAAAAAT", "ACTGCAGGGCA"))

def probability(l):
    return(math.exp(l)/(1+math.exp(l)))

#### Main function generating dataframe with scored guides

In [35]:
def spliceR_main(
    genomic_features,
    coding_features,
    genomic_sequence,
    coding_sequence,
    motif_weights_dict,
    position_weights_dict,
    output_as_df = True,
    haplotype_name = None,
    guide_length = 20, 
    pam = "NGG",
    flank_length = 20,
    CBE = True, 
    ABE = True,
    SA = True, 
    SD = True,
    logistic_adjust = 1, 
    max_exon_targeted = None # for certain HLAs, we want to stop considering later exons as potential targets since they encode cytoplasmic tail/DQB1 has exon 5 splice acceptor polymorphisms which cause problems w the code 
):
    """ 
    inputs: 
        - genomic_features: a list of dictionaries, one dict for each utr/exon/intron, keys are 'length', 'start', 'type', 'number' (for intron/exons only)
        partial example: [{'length': 300, 'start': 1, 'type': '5utr'},
        {'length': 73,
        'number': '1',
        'partial': False,
        'start': 301,
        'type': 'exon'},
        {'length': 130,
        'number': '1',
        'partial': False,
        'start': 374,
        'type': 'intron'}, ...
        - coding features: similar list of dictionaries as 'genomic_features' but only includes exons 
        - genomic sequence: string of genomic DNA sequence 
        - coding sequence: string of cDNA sequence 
    
    returns: dataframe (or list of lists) with columns: 
    ['haplotype name', 'exon number', 'splice site type', 'protospacer', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct']
    """
    ### inner helper functions for getting CBE/ABE guides for SA and SD sites 

    def get_CBE_SD_guides(haplotype_name, exon_number, pam, exon_end_idx, flank_length, genomic_sequence, position_weights_dict, motif_weights_dict, guide_length, pct_of_cds):
        pam_length = len(pam)
        sd_start = exon_end_idx-flank_length-pam_length
        sd_end = exon_end_idx+flank_length+pam_length
        SD_seq = get_rev_complement(genomic_sequence[sd_start:sd_end])
        guide_template_w_pam = "N"*guide_length+pam
        unscored_CBE_SD_guides = matchDNAPattern(guide_template_w_pam, SD_seq) # 0 indexed 
        motif = get_rev_complement(genomic_sequence[exon_end_idx-3:exon_end_idx+4])
        output = []
        for u_cbe_sd_guide in unscored_CBE_SD_guides:
            strt = u_cbe_sd_guide[0]
            target_pos = flank_length+len(pam)-strt #position of target base within the guide (1 to 20 + pam)
            if 1<=target_pos<=20:
                pw = position_weights_dict["CBE"][target_pos]['position_weight']
                mw = motif_weights_dict[motif]['motif_weight']
                cbe_score = probability(mw+pw-logistic_adjust)
                u_cbe_sd_guide_wo_pam = u_cbe_sd_guide[1][:-3]
                guide_info = [haplotype_name, exon_number, "donor",u_cbe_sd_guide_wo_pam, u_cbe_sd_guide[1], pam, "CBE", target_pos, cbe_score, pct_of_cds]
                output.append(guide_info)
        return output

    def get_ABE_SD_guides(haplotype_name, exon_number, pam, exon_end_idx, flank_length, genomic_sequence, position_weights_dict, motif_weights_dict, guide_length, pct_of_cds):
        pam_length = len(pam)
        sd_start = exon_end_idx-flank_length-pam_length
        sd_end = exon_end_idx+flank_length+pam_length
        SD_seq = get_rev_complement(genomic_sequence[sd_start:sd_end])
        guide_template_w_pam = "N"*guide_length+pam
        unscored_ABE_SD_guides = matchDNAPattern(guide_template_w_pam, SD_seq) # 0 indexed 
        motif = get_rev_complement(genomic_sequence[exon_end_idx-2:exon_end_idx+5])
        output = []
        for u_abe_sd_guide in unscored_ABE_SD_guides:
            strt = u_abe_sd_guide[0]
            target_pos = flank_length+len(pam)-strt-1 #position of target base within the guide (1 to 20 + pam)
            if 1<=target_pos<=20:
                pw = position_weights_dict["ABE"][target_pos]['position_weight']
                mw = motif_weights_dict[motif]['motif_weight']
                abe_score = probability(mw+pw-logistic_adjust)
                u_abe_sd_guide_wo_pam = u_abe_sd_guide[1][:-3]
                guide_info = [haplotype_name, exon_number, "donor", u_abe_sd_guide_wo_pam, u_abe_sd_guide[1], pam, "ABE", target_pos, abe_score, pct_of_cds]
                output.append(guide_info)
        return output

    def get_CBE_SA_guides(haplotype_name, exon_number, pam, exon_start_idx, flank_length, genomic_sequence, position_weights_dict, motif_weights_dict, guide_length, pct_of_cds):
        pam_length = len(pam)
        sa_start = exon_start_idx-flank_length-pam_length
        sa_end = exon_start_idx+flank_length+pam_length
        SA_seq = get_rev_complement(genomic_sequence[sa_start:sa_end])
        guide_template_w_pam = "N"*guide_length+pam
        unscored_CBE_SA_guides = matchDNAPattern(guide_template_w_pam, SA_seq)
        motif = get_rev_complement(genomic_sequence[exon_start_idx-4:exon_start_idx+3]) #checked 
        # scoring all potential guides
        output = []
        for u_cbe_sa_guide in unscored_CBE_SA_guides: 
            strt = u_cbe_sa_guide[0]
            target_pos = flank_length+len(pam)-strt+1 #position of target base within the guide (1 to 20 + pam) checked
            if 1<=target_pos<=20:
                pw = position_weights_dict["CBE"][target_pos]['position_weight']
                mw = motif_weights_dict[motif]['motif_weight']
                cbe_score = probability(mw+pw-logistic_adjust)
                u_cbe_sa_guide_wo_pam = u_cbe_sa_guide[1][:-3]
                guide_info = [haplotype_name, exon_number, "acceptor", u_cbe_sa_guide_wo_pam, u_cbe_sa_guide[1], pam, "CBE", target_pos, cbe_score, pct_of_cds]
                output.append(guide_info)
        return output

    def get_ABE_SA_guides(haplotype_name, exon_number, pam, exon_start_idx, flank_length, genomic_sequence, position_weights_dict, motif_weights_dict, guide_length, pct_of_cds):
        # unlike the other 3 cases, SA seq to be edited is NOT reverse complement 
        pam_length = len(pam)
        sa_start = exon_start_idx-flank_length-pam_length
        sa_end = exon_start_idx+flank_length+pam_length
        guide_template_w_pam = "N"*guide_length+pam
        SA_seq = genomic_sequence[sa_start:sa_end]
        unscored_ABE_SA_guides = matchDNAPattern(guide_template_w_pam, SA_seq)
        motif = genomic_sequence[exon_start_idx-5:exon_start_idx+2] #checked
        # scoring all potential guides
        output=[]
        for u_abe_sa_guide in unscored_ABE_SA_guides: 
            strt = u_abe_sa_guide[0]
            target_pos = flank_length+len(pam)-strt-1 #position of target base within the guide (1 to 20 + pam) checked
            if 1<=target_pos<=20:
                pw = position_weights_dict["ABE"][target_pos]['position_weight']
                mw = motif_weights_dict[motif]['motif_weight']
                abe_score = probability(mw+pw-logistic_adjust)
                u_abe_sa_guide_wo_pam = u_abe_sa_guide[1][:-3]
                guide_info = [haplotype_name, exon_number, "acceptor", u_abe_sa_guide_wo_pam, u_abe_sa_guide[1], pam, "ABE", target_pos, abe_score, pct_of_cds]
                output.append(guide_info)
        return output
    
    #### MAIN FUNCTION 
    cols = ['haplotype name', 'exon number', 'splice site type', 'protospacer', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct']

    exons_list = []
    num_exons = 0 
    for feature in genomic_features:
        if feature['type'] =='exon':
            if max_exon_targeted is not None:
                if int(feature['number']) <= max_exon_targeted:
                    exons_list.append(feature)
            else: exons_list.append(feature)
            if int(feature['number'])>num_exons:
                num_exons=int(feature['number'])

    print("num exons", num_exons)
    ## list to collect all outputs 
    output_list_of_lists = [] # each list corresponds to a possible guide 
    ## loop over all exons 
    for exon in exons_list:
        print("exon", exon['number'])

        ## calculating what percentage of the way through coding sequence the end of this exon is (want earlier disruption)
        end_of_exon_in_cds = 0 
        for cd_feat in coding_features:
            if cd_feat['number'] == exon['number']:
                end_of_exon_in_cds=cd_feat['start']+cd_feat['length']
                break
        pct_of_cds = end_of_exon_in_cds/len(coding_sequence)
        
        ## getting start and end coordinates of this exon (relative to GENOMIC sequence)
        exon_start_idx = exon['start']-1 #-1 to account for 0 indexing 
        exon_end_idx = exon_start_idx+exon['length']

        # check if it's first exon, if so, only get splice donor (after exon)
        if int(exon['number']) == 1:
            if SD:
                if CBE:
                    output_list_of_lists.extend(get_CBE_SD_guides(haplotype_name, exon['number'], pam, exon_end_idx, flank_length, genomic_sequence, position_weights_dict, motif_weights_dict, guide_length, pct_of_cds))
                
                if ABE:
                    output_list_of_lists.extend(get_ABE_SD_guides(haplotype_name, exon['number'], pam, exon_end_idx, flank_length, genomic_sequence, position_weights_dict, motif_weights_dict, guide_length, pct_of_cds))

        # check if it's last exon, if so, only get splice acceptor (before exon)
        elif int(exon['number']) == num_exons:
            # print("last exon", int(exon['number']))
            if SA:
                if CBE:
                    output_list_of_lists.extend(get_CBE_SA_guides(haplotype_name, exon['number'], pam, exon_start_idx, flank_length, genomic_sequence, position_weights_dict, motif_weights_dict, guide_length, pct_of_cds))

                if ABE: 
                    output_list_of_lists.extend(get_ABE_SA_guides(haplotype_name, exon['number'], pam, exon_start_idx, flank_length, genomic_sequence, position_weights_dict, motif_weights_dict, guide_length, pct_of_cds))
        
        else:
            # splice acceptor sites 
            if SA:
                if CBE:
                    output_list_of_lists.extend(get_CBE_SA_guides(haplotype_name, exon['number'], pam, exon_start_idx, flank_length, genomic_sequence, position_weights_dict, motif_weights_dict, guide_length, pct_of_cds))
                    print("SA, CBE")
                if ABE: 
                    output_list_of_lists.extend(get_ABE_SA_guides(haplotype_name, exon['number'], pam, exon_start_idx, flank_length, genomic_sequence, position_weights_dict, motif_weights_dict, guide_length, pct_of_cds))
                    print("SA, ABE")
            # splice donor sites 
            if SD:
                if CBE:
                    output_list_of_lists.extend(get_CBE_SD_guides(haplotype_name, exon['number'], pam, exon_end_idx, flank_length, genomic_sequence, position_weights_dict, motif_weights_dict, guide_length, pct_of_cds))
                    print("SD, CBE")
                if ABE:
                    output_list_of_lists.extend(get_ABE_SD_guides(haplotype_name, exon['number'], pam, exon_end_idx, flank_length, genomic_sequence, position_weights_dict, motif_weights_dict, guide_length, pct_of_cds))
                    print("SD, ABE")
    
    if output_as_df:
        return pd.DataFrame(output_list_of_lists, columns=cols)
    
    else: return output_list_of_lists
        

## For HLA project

### HLA-A

In [8]:
n_to_a = pd.read_csv("HLA_A_name_to_accession.txt", sep="\t")
n_to_a 
HLA_A_name_to_accession = dict(zip(n_to_a['name'], n_to_a['accession']))
HLA_A_name_to_accession
len(HLA_A_name_to_accession)
# name_to_accession #dict mapping HLA haplotype name to accession 

40

In [ ]:
#### running spliceR on all HLA A common haplotypes 

# initializing empty dataframe to store results 
cols = ['haplotype name', 'exon number', 'splice site type', 'protospacer', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct']
HLA_A_output_df = pd.DataFrame(columns=cols)

motif_weights_filepath = "./motif_weights.tsv"
position_weights_filepath = "./position_weights.tsv"
motif_weights_dict, position_weights_dict = parse_motif_position_weights(motif_weights_filepath, position_weights_filepath)

for haplotype in HLA_A_name_to_accession:
    print(haplotype)
    accession = HLA_A_name_to_accession[haplotype]
    hap_url = f"https://www.ebi.ac.uk/cgi-bin/ipd/api/allele/{accession}"
    # print(hp_url)
    hap_info = parse_json(file_url=hap_url)
    genomic_features = hap_info['genomic_features']
    coding_features = hap_info['coding_features']
    genomic_sequence = hap_info['genomic_sequence']
    coding_sequence = hap_info['sequences']['coding']

    hap_out = spliceR_main(
        genomic_features,
        coding_features,
        genomic_sequence,
        coding_sequence,
        motif_weights_dict,
        position_weights_dict,
        output_as_df = True,
        haplotype_name = haplotype,
        guide_length = 20, 
        pam = "NGG",
        flank_length = 20,
        CBE = True, 
        ABE = True,
        SA = True, 
        SD = True,
        logistic_adjust = 1
    )
    HLA_A_output_df = pd.concat([HLA_A_output_df, hap_out], axis=0, ignore_index=True)


HLA_A_output_df



A*30:04:01:01


/var/folders/sw/t72qsf8n6dj3nspvp9_rlmxr0000gp/T/ipykernel_57621/3182564382.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  HLA_A_output_df = pd.concat([HLA_A_output_df, hap_out], axis=0, ignore_index=True)


A*24:03:01:01
A*24:02:01:01
A*11:01:01:01
A*34:02:01:01
A*32:01:01:01
A*01:01:01:01
A*03:02:01:01
A*68:01:01:01
A*68:01:02:01
A*23:01:01:01
A*11:02:01:01
A*74:01:01:01
A*02:05:01:01
A*26:01:01:01
A*30:01:01:01
A*34:01:01:01
A*26:03:01:01
A*66:01:01:01
A*03:01:01:01
A*02:11:01:01
A*02:02:01:01
A*02:03:01:01
A*33:01:01:01
A*30:02:01:01
A*24:07:01:01
A*29:02:01:01
A*25:01:01:01
A*68:02:01:01
A*33:03:01:01
A*36:01:01:01
A*29:01:01:01
A*02:01:01:01
A*02:01:04
A*31:01:02:01
A*68:03:01:01
A*02:06:01:01
A*02:07:01:01
A*26:02:01
A*02:17:02:01


,haplotype name,exon number,splice site type,protospacer,guide seq with PAM,PAM,enzyme class,CBE/ABE position,guide score,cDNA distuption pct
0,A*30:04:01:01,1,donor,GACCCCGCACTCACCCGCCC,GACCCCGCACTCACCCGCCCAGG,NGG,CBE,14,0.113088,0.067395
1,A*30:04:01:01,1,donor,CGCACTCACCCGCCCAGGTC,CGCACTCACCCGCCCAGGTCTGG,NGG,CBE,9,0.334910,0.067395
2,A*30:04:01:01,1,donor,GCACTCACCCGCCCAGGTCT,GCACTCACCCGCCCAGGTCTGGG,NGG,CBE,8,0.466535,0.067395
3,A*30:04:01:01,1,donor,CACCCGCCCAGGTCTGGGTC,CACCCGCCCAGGTCTGGGTCAGG,NGG,CBE,3,0.336702,0.067395
4,A*30:04:01:01,1,donor,ACCCGCCCAGGTCTGGGTCA,ACCCGCCCAGGTCTGGGTCAGGG,NGG,CBE,2,0.257234,0.067395
...,...,...,...,...,...,...,...,...,...,...
2011,A*02:17:02:01,7,acceptor,CCCTGGGCACTGTCACTGCC,CCCTGGGCACTGTCACTGCCTGG,NGG,CBE,20,0.044522,0.996357
2012,A*02:17:02:01,7,acceptor,CCTGGGCACTGTCACTGCCT,CCTGGGCACTGTCACTGCCTGGG,NGG,CBE,19,0.059258,0.996357
2013,A*02:17:02:01,7,acceptor,CTGGGCACTGTCACTGCCTG,CTGGGCACTGTCACTGCCTGGGG,NGG,CBE,18,0.085840,0.996357
2014,A*02:17:02:01,7,acceptor,CCCAGGCAGTGACAGTGCCC,CCCAGGCAGTGACAGTGCCCAGG,NGG,ABE,4,0.238371,0.996357


In [ ]:
HLA_A_output_df.to_csv('HLA_A_all_haps_splice_site_BE.csv', index=False)

### HLA-B

In [ ]:
hla_b_n_to_a = pd.read_csv("HLA_B_name_to_accession.txt", sep="\t")
hla_b_n_to_a 
HLA_B_name_to_accession = dict(zip(hla_b_n_to_a['name'], hla_b_n_to_a['accession']))
HLA_B_name_to_accession
len(HLA_B_name_to_accession)

## there are 76 different haplotypes to target b/c ig 2 of the original list of 74 have 2 versions that are both marked as common?? 

76

In [ ]:
#### running spliceR on all HLA B common haplotypes 

# initializing empty dataframe to store results 
cols = ['haplotype name', 'exon number', 'splice site type', 'protospacer', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct']
HLA_B_output_df = pd.DataFrame(columns=cols)

motif_weights_filepath = "./motif_weights.tsv"
position_weights_filepath = "./position_weights.tsv"
motif_weights_dict, position_weights_dict = parse_motif_position_weights(motif_weights_filepath, position_weights_filepath)

for haplotype in HLA_B_name_to_accession:
    print(haplotype)
    accession = HLA_B_name_to_accession[haplotype]
    hap_url = f"https://www.ebi.ac.uk/cgi-bin/ipd/api/allele/{accession}"
    # print(hp_url)
    hap_info = parse_json(file_url=hap_url)
    genomic_features = hap_info['genomic_features']
    coding_features = hap_info['coding_features']
    genomic_sequence = hap_info['genomic_sequence']
    coding_sequence = hap_info['sequences']['coding']

    hap_out = spliceR_main(
        genomic_features,
        coding_features,
        genomic_sequence,
        coding_sequence,
        motif_weights_dict,
        position_weights_dict,
        output_as_df = True,
        haplotype_name = haplotype,
        guide_length = 20, 
        pam = "NGG",
        flank_length = 20,
        CBE = True, 
        ABE = True,
        SA = True, 
        SD = True,
        logistic_adjust = 1
    )
    HLA_B_output_df = pd.concat([HLA_B_output_df, hap_out], axis=0, ignore_index=True)


HLA_B_output_df

B*54:01:01:01


/var/folders/sw/t72qsf8n6dj3nspvp9_rlmxr0000gp/T/ipykernel_81124/2423316822.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  HLA_B_output_df = pd.concat([HLA_B_output_df, hap_out], axis=0, ignore_index=True)


B*55:01:01:01
B*59:01:01:01
B*44:02:01:01
B*81:01:01:01
B*13:02:01:01
B*07:02:01:01
B*07:05:01:01
B*50:01:01:01
B*35:12:01:01
B*51:06:01:01
B*13:01:01:01
B*41:01:01:01
B*15:11:01:01
B*39:06:02:01
B*57:01:01:01
B*44:03:01:01
B*44:03:02:01
B*51:02:01:01
B*15:35:01:01
B*56:01:01:01
B*15:16:01:01
B*57:03:01:01
B*27:04:01
B*35:01:01:01
B*27:05:02:01
B*58:02:01:01
B*58:01:01:01
B*42:01:01:01
B*15:01:01:01
B*15:15:01:01
B*39:10:01
B*15:03:01:01
B*14:02:01:01
B*49:01:01:01
B*35:02:01:01
B*51:01:01:01
B*46:01:01:01
B*35:08:01:01
B*15:21:01:01
B*55:02:01:01
B*40:02:01:01
B*15:13:01
B*35:17:01:01
B*53:01:01:01
B*15:10:01:01
B*38:01:01:01
B*15:12:01
B*37:01:01:01
B*35:43:01
B*27:06:01:01
B*35:03:01:01
B*18:01:01:01
B*48:01:01:01
B*35:05:01:01
B*15:17:01:01
B*40:06:01:01
B*52:01:01:01
B*52:01:02:01
B*15:18:01:01
B*14:01:01:01
B*38:02:01:01
B*39:01:01:01
B*15:25:01:01
B*39:05:01:01
B*08:01:01:01
B*45:01:01:01
B*67:01:01
B*15:02:01:01
B*40:01:02:01
B*15:07:01:01
B*40:10:01:01
B*15:06
B*56:02:01:01
B*

,haplotype name,exon number,splice site type,protospacer,guide seq with PAM,PAM,enzyme class,CBE/ABE position,guide score,cDNA distuption pct
0,B*54:01:01:01,1,donor,CCCTCCCGACCCGCACTCAC,CCCTCCCGACCCGCACTCACCGG,NGG,CBE,20,0.044239,0.067952
1,B*54:01:01:01,1,donor,CGACCCGCACTCACCGGCCC,CGACCCGCACTCACCGGCCCAGG,NGG,CBE,14,0.143533,0.067952
2,B*54:01:01:01,1,donor,GCACTCACCGGCCCAGGTCT,GCACTCACCGGCCCAGGTCTCGG,NGG,CBE,8,0.534763,0.067952
3,B*54:01:01:01,1,donor,CACCGGCCCAGGTCTCGGTC,CACCGGCCCAGGTCTCGGTCAGG,NGG,CBE,3,0.400186,0.067952
4,B*54:01:01:01,1,donor,ACCGGCCCAGGTCTCGGTCA,ACCGGCCCAGGTCTCGGTCAGGG,NGG,CBE,2,0.312800,0.067952
...,...,...,...,...,...,...,...,...,...,...
4208,B*15:27:01:01,6,acceptor,TTCCCACAGGTGGAAAAGGA,TTCCCACAGGTGGAAAAGGAGGG,NGG,ABE,8,0.254955,0.960514
4209,B*15:27:01:01,7,acceptor,CCCTGGGCACTGTCGCTGGC,CCCTGGGCACTGTCGCTGGCTGG,NGG,CBE,20,0.006267,1.000918
4210,B*15:27:01:01,7,acceptor,GGCTGGAGTAGAACAAAAAC,GGCTGGAGTAGAACAAAAACAGG,NGG,CBE,3,0.083324,1.000918
4211,B*15:27:01:01,7,acceptor,TCCAGCCAGCGACAGTGCCC,TCCAGCCAGCGACAGTGCCCAGG,NGG,ABE,4,0.145725,1.000918


In [22]:
HLA_B_output_df.to_csv('HLA_B_all_haps_splice_site_BE.csv', index=False)

### HLA-C 

In [ ]:
hla_c_n_to_a = pd.read_csv("HLA_C_name_to_accession.txt", sep="\t")
hla_c_n_to_a 
HLA_C_name_to_accession = dict(zip(hla_c_n_to_a['name'], hla_c_n_to_a['accession']))
HLA_C_name_to_accession
len(HLA_C_name_to_accession) # 35 target haplotypes 

In [ ]:
#### running spliceR on all HLA C common haplotypes 

# initializing empty dataframe to store results 
cols = ['haplotype name', 'exon number', 'splice site type', 'protospacer', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct']
HLA_C_output_df = pd.DataFrame(columns=cols)

motif_weights_filepath = "./motif_weights.tsv"
position_weights_filepath = "./position_weights.tsv"
motif_weights_dict, position_weights_dict = parse_motif_position_weights(motif_weights_filepath, position_weights_filepath)

for haplotype in HLA_C_name_to_accession:
    print(haplotype)
    accession = HLA_C_name_to_accession[haplotype]
    hap_url = f"https://www.ebi.ac.uk/cgi-bin/ipd/api/allele/{accession}"
    # print(hp_url)
    hap_info = parse_json(file_url=hap_url)
    genomic_features = hap_info['genomic_features']
    coding_features = hap_info['coding_features']
    genomic_sequence = hap_info['genomic_sequence']
    coding_sequence = hap_info['sequences']['coding']

    hap_out = spliceR_main(
        genomic_features,
        coding_features,
        genomic_sequence,
        coding_sequence,
        motif_weights_dict,
        position_weights_dict,
        output_as_df = True,
        haplotype_name = haplotype,
        guide_length = 20, 
        pam = "NGG",
        flank_length = 20,
        CBE = True, 
        ABE = True,
        SA = True, 
        SD = True,
        logistic_adjust = 1
    )
    HLA_C_output_df = pd.concat([HLA_C_output_df, hap_out], axis=0, ignore_index=True)


HLA_C_output_df

C*07:02:01:01


/var/folders/sw/t72qsf8n6dj3nspvp9_rlmxr0000gp/T/ipykernel_81124/1427478302.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  HLA_C_output_df = pd.concat([HLA_C_output_df, hap_out], axis=0, ignore_index=True)


C*14:02:01:01
C*14:02:03
C*07:04:01:01
C*08:04:01:01
C*08:01:01:01
C*15:02:01:01
C*03:04:01:01
C*03:04:02:01
C*12:04:02:01
C*03:02:01
C*08:03:01:01
C*04:01:01:01
C*07:01:01:01
C*16:01:01:01
C*06:02:01:01
C*05:01:01:01
C*03:03:01:01
C*03:03:04
C*12:02:02:01
C*03:06:01:01
C*16:02:01:01
C*01:02:01:01
C*03:05:01:01
C*14:03:01:01
C*02:02:02:01
C*16:04:01:01
C*18:01:01:01
C*04:03:01:01
C*15:05:01:01
C*12:03:01:01
C*08:02:01:01
C*08:06
C*17:01:01:02
C*04:06:01


,haplotype name,exon number,splice site type,protospacer,guide seq with PAM,PAM,enzyme class,CBE/ABE position,guide score,cDNA distuption pct
0,C*07:02:01:01,1,donor,CCTCCCAACCCCGCACTCAC,CCTCCCAACCCCGCACTCACAGG,NGG,CBE,20,0.034682,0.067212
1,C*07:02:01:01,1,donor,AACCCCGCACTCACAGGCCC,AACCCCGCACTCACAGGCCCAGG,NGG,CBE,14,0.115109,0.067212
2,C*07:02:01:01,1,donor,GCACTCACAGGCCCAGGTCT,GCACTCACAGGCCCAGGTCTCGG,NGG,CBE,8,0.471516,0.067212
3,C*07:02:01:01,1,donor,CACAGGCCCAGGTCTCGGTC,CACAGGCCCAGGTCTCGGTCAGG,NGG,CBE,3,0.341184,0.067212
4,C*07:02:01:01,1,donor,ACAGGCCCAGGTCTCGGTCA,ACAGGCCCAGGTCTCGGTCAGGG,NGG,CBE,2,0.261074,0.067212
...,...,...,...,...,...,...,...,...,...,...
1949,C*04:06:01,7,acceptor,CCCTGGGCACTGTTGCTGGC,CCCTGGGCACTGTTGCTGGCTGG,NGG,CBE,20,0.006267,0.996367
1950,C*04:06:01,7,acceptor,CCTGGGCACTGTTGCTGGCT,CCTGGGCACTGTTGCTGGCTGGG,NGG,CBE,19,0.008453,0.996367
1951,C*04:06:01,7,acceptor,CTGGGCACTGTTGCTGGCTG,CTGGGCACTGTTGCTGGCTGGGG,NGG,CBE,18,0.012549,0.996367
1952,C*04:06:01,7,acceptor,CCCAGCCAGCAACAGTGCCC,CCCAGCCAGCAACAGTGCCCAGG,NGG,ABE,4,0.188218,0.996367


In [ ]:
## saving HLA C raw outputs 
HLA_C_output_df.to_csv('HLA_C_all_haps_splice_site_BE.csv', index=False)

### HLA-DQB1

In [7]:
hla_dqb1_n_to_a = pd.read_csv("HLA_name_to_dbAccession/HLA_DQB1_name_to_accession.txt", sep="\t")
hla_dqb1_n_to_a
HLA_DQB1_name_to_accession = dict(zip(hla_dqb1_n_to_a['name'], hla_dqb1_n_to_a['accession']))
HLA_DQB1_name_to_accession
# len(HLA_DQB1_name_to_accession) # 14 target haplotypes 

{'DQB1*03:02:01:01': 'HLA00627',
 'DQB1*02:01:01:01': 'HLA00622',
 'DQB1*05:02:01:01': 'HLA00639',
 'DQB1*03:01:01:01': 'HLA00625',
 'DQB1*05:03:01:01': 'HLA00640',
 'DQB1*06:04:01:01': 'HLA00648',
 'DQB1*04:02:01:01': 'HLA00637',
 'DQB1*06:09:01:01': 'HLA00654',
 'DQB1*06:02:01:01': 'HLA00646',
 'DQB1*03:03:02:01': 'HLA00629',
 'DQB1*04:01:01:01': 'HLA00636',
 'DQB1*06:01:01:01': 'HLA00643',
 'DQB1*05:01:01:01': 'HLA00638',
 'DQB1*06:03:01:01': 'HLA00647'}

In [ ]:
#### running spliceR on all HLA DQB1 common haplotypes 

# initializing empty dataframe to store results 
cols = ['haplotype name', 'exon number', 'splice site type', 'protospacer', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct']
HLA_DQB1_output_df = pd.DataFrame(columns=cols)

motif_weights_filepath = "./motif_weights.tsv"
position_weights_filepath = "./position_weights.tsv"
motif_weights_dict, position_weights_dict = parse_motif_position_weights(motif_weights_filepath, position_weights_filepath)

for haplotype in HLA_DQB1_name_to_accession:
    print(haplotype)
    accession = HLA_DQB1_name_to_accession[haplotype]
    hap_url = f"https://www.ebi.ac.uk/cgi-bin/ipd/api/allele/{accession}"
    hap_info = parse_json(file_url=hap_url)
    genomic_features = hap_info['genomic_features']
    coding_features = hap_info['coding_features']
    genomic_sequence = hap_info['genomic_sequence']
    coding_sequence = hap_info['sequences']['coding']

    print(genomic_features)

    hap_out = spliceR_main(
        genomic_features,
        coding_features,
        genomic_sequence,
        coding_sequence,
        motif_weights_dict,
        position_weights_dict,
        output_as_df = True,
        haplotype_name = haplotype,
        guide_length = 20, 
        pam = "NGG",
        flank_length = 20,
        CBE = True, 
        ABE = True,
        SA = True, 
        SD = True,
        logistic_adjust = 1, 
        max_exon_targeted = 4 #HLA-DQB1 has exon 5 splice acceptor site polymorphisms which causes issues with the code since the motif is not the typical SA site motif (exons 5, 6 are intracellular anyway so are not good targets)
    )
    HLA_DQB1_output_df = pd.concat([HLA_DQB1_output_df, hap_out], axis=0, ignore_index=True)


HLA_DQB1_output_df

In [38]:
## saving HLA DQB1 raw outputs 
HLA_DQB1_output_df.to_csv('HLA_DQB1_all_haps_splice_site_BE_exclude_exons5_6.csv', index=False)

### HLA-DRB1

In [39]:
hla_drb1_n_to_a = pd.read_csv("HLA_name_to_dbAccession/HLA_DRB1_name_to_accession.txt", sep="\t")
hla_drb1_n_to_a
HLA_DRB1_name_to_accession = dict(zip(hla_drb1_n_to_a['name'], hla_drb1_n_to_a['accession']))
HLA_DRB1_name_to_accession

{'DRB1*14:05:01:01': 'HLA00837',
 'DRB1*14:04:01:01': 'HLA00836',
 'DRB1*15:02:02:01': 'HLA00868',
 'DRB1*04:05:01:01': 'HLA00690',
 'DRB1*08:02:01:01': 'HLA00724',
 'DRB1*04:07:01:01': 'HLA00693',
 'DRB1*15:06:01': 'HLA00873',
 'DRB1*13:02:01:01': 'HLA00798',
 'DRB1*09:01:02:01': 'HLA00749',
 'DRB1*08:04:01:01': 'HLA00728',
 'DRB1*08:04:04': 'HLA01414',
 'DRB1*11:04:01:01': 'HLA00756',
 'DRB1*11:04:02': 'HLA00757',
 'DRB1*04:06:01': 'HLA00692',
 'DRB1*15:03:01:01': 'HLA00870',
 'DRB1*13:03:01:01': 'HLA00799',
 'DRB1*04:02:01': 'HLA00687',
 'DRB1*12:01:01:01': 'HLA00789',
 'DRB1*04:01:01:01': 'HLA00685',
 'DRB1*01:02:01:01': 'HLA00665',
 'DRB1*11:01:01:01': 'HLA00751',
 'DRB1*11:01:02:01': 'HLA00752',
 'DRB1*14:02:01:01': 'HLA00834',
 'DRB1*15:01:01:01': 'HLA00865',
 'DRB1*03:02:01:01': 'HLA00673',
 'DRB1*14:01:01:01': 'HLA00833',
 'DRB1*03:01:01:01': 'HLA00671',
 'DRB1*01:01:01:01': 'HLA00664',
 'DRB1*01:01:02': 'HLA01598',
 'DRB1*12:02:01:01': 'HLA00790',
 'DRB1*12:02:02:01': 'HLA007

In [40]:
#### running spliceR on all HLA DRB1 common haplotypes 

# initializing empty dataframe to store results 
cols = ['haplotype name', 'exon number', 'splice site type', 'protospacer', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct']
HLA_DRB1_output_df = pd.DataFrame(columns=cols)

motif_weights_filepath = "./motif_weights.tsv"
position_weights_filepath = "./position_weights.tsv"
motif_weights_dict, position_weights_dict = parse_motif_position_weights(motif_weights_filepath, position_weights_filepath)

for haplotype in HLA_DRB1_name_to_accession:
    print(haplotype)
    accession = HLA_DRB1_name_to_accession[haplotype]
    hap_url = f"https://www.ebi.ac.uk/cgi-bin/ipd/api/allele/{accession}"
    hap_info = parse_json(file_url=hap_url)
    genomic_features = hap_info['genomic_features']
    coding_features = hap_info['coding_features']
    genomic_sequence = hap_info['genomic_sequence']
    coding_sequence = hap_info['sequences']['coding']

    hap_out = spliceR_main(
        genomic_features,
        coding_features,
        genomic_sequence,
        coding_sequence,
        motif_weights_dict,
        position_weights_dict,
        output_as_df = True,
        haplotype_name = haplotype,
        guide_length = 20, 
        pam = "NGG",
        flank_length = 20,
        CBE = True, 
        ABE = True,
        SA = True, 
        SD = True,
        logistic_adjust = 1, 
        max_exon_targeted = 4 #exons 5, 6 are intracellular so are not good targets
    )
    HLA_DRB1_output_df = pd.concat([HLA_DRB1_output_df, hap_out], axis=0, ignore_index=True)


HLA_DRB1_output_df

DRB1*14:05:01:01
num exons 6
exon 1
exon 2
SA, CBE
SA, ABE
SD, CBE
SD, ABE
exon 3
SA, CBE
SA, ABE
SD, CBE
SD, ABE
exon 4
SA, CBE
SA, ABE
SD, CBE
SD, ABE
DRB1*14:04:01:01


/var/folders/sm/2bd25dyd1bx5nl6h9b6j4z7c0000gn/T/ipykernel_14360/1043908064.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  HLA_DRB1_output_df = pd.concat([HLA_DRB1_output_df, hap_out], axis=0, ignore_index=True)


num exons 6
exon 1
exon 2
SA, CBE
SA, ABE
SD, CBE
SD, ABE
exon 3
SA, CBE
SA, ABE
SD, CBE
SD, ABE
exon 4
SA, CBE
SA, ABE
SD, CBE
SD, ABE
DRB1*15:02:02:01
num exons 6
exon 1
exon 2
SA, CBE
SA, ABE
SD, CBE
SD, ABE
exon 3
SA, CBE
SA, ABE
SD, CBE
SD, ABE
exon 4
SA, CBE
SA, ABE
SD, CBE
SD, ABE
DRB1*04:05:01:01
num exons 6
exon 1
exon 2
SA, CBE
SA, ABE
SD, CBE
SD, ABE
exon 3
SA, CBE
SA, ABE
SD, CBE
SD, ABE
exon 4
SA, CBE
SA, ABE
SD, CBE
SD, ABE
DRB1*08:02:01:01
num exons 6
exon 1
exon 2
SA, CBE
SA, ABE
SD, CBE
SD, ABE
exon 3
SA, CBE
SA, ABE
SD, CBE
SD, ABE
exon 4
SA, CBE
SA, ABE
SD, CBE
SD, ABE
DRB1*04:07:01:01
num exons 6
exon 1
exon 2
SA, CBE
SA, ABE
SD, CBE
SD, ABE
exon 3
SA, CBE
SA, ABE
SD, CBE
SD, ABE
exon 4
SA, CBE
SA, ABE
SD, CBE
SD, ABE
DRB1*15:06:01
num exons 6
exon 1
exon 2
SA, CBE
SA, ABE
SD, CBE
SD, ABE
exon 3
SA, CBE
SA, ABE
SD, CBE
SD, ABE
exon 4
SA, CBE
SA, ABE
SD, CBE
SD, ABE
DRB1*13:02:01:01
num exons 6
exon 1
exon 2
SA, CBE
SA, ABE
SD, CBE
SD, ABE
exon 3
SA, CBE
SA, ABE
SD, 

,haplotype name,exon number,splice site type,protospacer,guide seq with PAM,PAM,enzyme class,CBE/ABE position,guide score,cDNA disruption pct
0,DRB1*14:05:01:01,1,donor,CCACAATGTGCACTTACGTC,CCACAATGTGCACTTACGTCTGG,NGG,CBE,17,0.048857,0.126092
1,DRB1*14:05:01:01,1,donor,CCACAATGTGCACTTACGTC,CCACAATGTGCACTTACGTCTGG,NGG,ABE,16,0.045911,0.126092
2,DRB1*14:05:01:01,2,acceptor,GTACTCCAAGAAACGTGCTG,GTACTCCAAGAAACGTGCTGTGG,NGG,CBE,18,0.012222,0.463171
3,DRB1*14:05:01:01,2,acceptor,TACTCCAAGAAACGTGCTGT,TACTCCAAGAAACGTGCTGTGGG,NGG,CBE,17,0.020726,0.463171
4,DRB1*14:05:01:01,2,acceptor,ACTCCAAGAAACGTGCTGTG,ACTCCAAGAAACGTGCTGTGGGG,NGG,CBE,16,0.024067,0.463171
...,...,...,...,...,...,...,...,...,...,...
951,DRB1*14:08,3,donor,TCACTCCATTCCACTGTGAG,TCACTCCATTCCACTGTGAGAGG,NGG,ABE,3,0.144056,0.815231
952,DRB1*14:08,3,donor,CACTCCATTCCACTGTGAGA,CACTCCATTCCACTGTGAGAGGG,NGG,ABE,2,0.075413,0.815231
953,DRB1*14:08,4,acceptor,TCAGACCGTGCTCCTGAGAG,TCAGACCGTGCTCCTGAGAGAGG,NGG,CBE,14,0.116474,0.953808
954,DRB1*14:08,4,acceptor,TGCTCCTGAGAGAGGAAGCC,TGCTCCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.843209,0.953808


In [41]:
## saving HLA DQB1 raw outputs 
HLA_DRB1_output_df.to_csv('HLA_DRB1_all_haps_splice_site_BE_exclude_exons5_6.csv', index=False)

### debugging

In [30]:
exons_list = []
num_exons = 0 
for feature in genomic_features:
    if feature['type'] =='exon':
        exons_list.append(feature)
        if int(feature['number'])>num_exons:
            num_exons=int(feature['number'])

exons_list
print("exons list", exons_list)

pam="NGG"
flank_length=7
guide_length=5
haplotype_name= "DQB1*02:01:01:01"
logistic_adjust = 1
pct_of_cds = 0

for i, exon in enumerate(exons_list):
    exon_number = i
    print("exon", exon['number'])

    ## calculating what percentage of the way through coding sequence the end of this exon is (want earlier disruption)
    # end_of_exon_in_cds = 0 
    # for cd_feat in coding_features:
    #     if cd_feat['number'] == exon['number']:
    #         end_of_exon_in_cds=cd_feat['start']+cd_feat['length']
    #         break
    # pct_of_cds = end_of_exon_in_cds/len(coding_sequence)
    
    ## getting start and end coordinates of this exon (relative to GENOMIC sequence)
    exon_start_idx = exon['start']-1 #-1 to account for 0 indexing 
    exon_end_idx = exon_start_idx+exon['length']
    
    print("exon start", exon_start_idx)
    print("start of exon", genomic_sequence[exon_start_idx:exon_start_idx+5])
    pam_length = len(pam)
    sa_start = exon_start_idx-flank_length-pam_length
    sa_end = exon_start_idx+flank_length+pam_length

    print("sa_start", sa_start)
    print("sa_end", sa_end)

    SA_seq = get_rev_complement(genomic_sequence[sa_start:sa_end])
    guide_template_w_pam = "N"*guide_length+pam
    unscored_CBE_SA_guides = matchDNAPattern(guide_template_w_pam, SA_seq)
    motif = get_rev_complement(genomic_sequence[exon_start_idx-4:exon_start_idx+3]) #checked 

    print("sa_seq", SA_seq)
    print("motif", motif)

    output = []
    for u_cbe_sd_guide in unscored_CBE_SA_guides:
        strt = u_cbe_sd_guide[0]
        target_pos = flank_length+len(pam)-strt #position of target base within the guide (1 to 20 + pam)
        if 1<=target_pos<=20:
            pw = position_weights_dict["CBE"][target_pos]['position_weight']
            mw = motif_weights_dict[motif]['motif_weight']
            cbe_score = probability(mw+pw-logistic_adjust)
            u_cbe_sd_guide_wo_pam = u_cbe_sd_guide[1][:-3]
            guide_info = [haplotype_name, exon_number, "donor",u_cbe_sd_guide_wo_pam, u_cbe_sd_guide[1], pam, "CBE", target_pos, cbe_score, pct_of_cds]
            output.append(guide_info)


exons list [{'length': 109, 'number': '1', 'partial': False, 'start': 530, 'type': 'exon'}, {'length': 270, 'number': '2', 'partial': False, 'start': 2083, 'type': 'exon'}, {'length': 282, 'number': '3', 'partial': False, 'start': 4883, 'type': 'exon'}, {'length': 111, 'number': '4', 'partial': False, 'start': 5680, 'type': 'exon'}, {'length': 24, 'number': '5', 'partial': False, 'start': 6273, 'type': 'exon'}, {'length': 14, 'number': '6', 'partial': False, 'start': 6907, 'type': 'exon'}]
exon 1
exon start 529
start of exon ATGTC
sa_start 519
sa_end 539
sa_seq TCCAAGACATAATTGAGACG
motif CATAATT
exon 2
exon start 2082
start of exon AGGAT
sa_start 2072
sa_end 2092
sa_seq ACGAAATCCTCTGCGGGGAA
motif CCTCTGC
exon 3
exon start 4882
start of exon TGGAG
sa_start 4872
sa_end 4892
sa_seq GTGGGCTCCACTGAGGGCAG
motif CCACTGA
exon 4
exon start 5679
start of exon GGGCT
sa_start 5669
sa_end 5689
sa_seq GACTGAGCCCCTAAGAAGCA
motif CCCCTAA
exon 5
exon start 6272
start of exon GACCT
sa_start 6262
sa_end 

KeyError: 'GTCTTAC'

In [ ]:
def get_CBE_SA_guides(haplotype_name, exon_number, pam, exon_start_idx, flank_length, genomic_sequence, position_weights_dict, motif_weights_dict, guide_length, pct_of_cds):
    pam_length = len(pam)
    sa_start = exon_start_idx-flank_length-pam_length
    sa_end = exon_start_idx+flank_length+pam_length
    SA_seq = get_rev_complement(genomic_sequence[sa_start:sa_end])
    guide_template_w_pam = "N"*guide_length+pam
    unscored_CBE_SA_guides = matchDNAPattern(guide_template_w_pam, SA_seq)
    motif = get_rev_complement(genomic_sequence[exon_start_idx-4:exon_start_idx+3]) #checked 
    # scoring all potential guides
    output = []
    for u_cbe_sa_guide in unscored_CBE_SA_guides: 
        strt = u_cbe_sa_guide[0]
        target_pos = flank_length+len(pam)-strt+1 #position of target base within the guide (1 to 20 + pam) checked
        if 1<=target_pos<=20:
            pw = position_weights_dict["CBE"][target_pos]['position_weight']
            mw = motif_weights_dict[motif]['motif_weight']
            cbe_score = probability(mw+pw-logistic_adjust)
            u_cbe_sa_guide_wo_pam = u_cbe_sa_guide[1][:-3]
            guide_info = [haplotype_name, exon_number, "acceptor", u_cbe_sa_guide_wo_pam, u_cbe_sa_guide[1], pam, "CBE", target_pos, cbe_score, pct_of_cds]
            output.append(guide_info)
    return output

In [10]:
motif_weights_dict, position_weights_dict = parse_motif_position_weights(motif_weights_filepath, position_weights_filepath)
motif_weights_dict['GTCTTAC']

KeyError: 'GTCTTAC'

## tests/extras

In [15]:
# for HLA project: 
motif_weights_filepath = "../motif_weights.tsv"
position_weights_filepath = "../position_weights.tsv"
motif_weights_dict, position_weights_dict = parse_motif_position_weights(motif_weights_filepath, position_weights_filepath)

A_24_02_info = parse_json("HLA00050.json")
genomic_features = A_24_02_info['genomic_features']
coding_features = A_24_02_info['coding_features']
genomic_sequence = A_24_02_info['genomic_sequence']
coding_sequence = A_24_02_info['sequences']['coding']

test1 = spliceR_main(
    genomic_features,
    coding_features,
    genomic_sequence,
    coding_sequence,
    motif_weights_dict,
    position_weights_dict,
    output_as_df = True,
    haplotype_name = "A*24:02:01:01",
    guide_length = 20, 
    pam = "NGG",
    flank_length = 20,
    CBE = True, 
    ABE = True,
    SA = True, 
    SD = True,
    logistic_adjust = 1
)
test1

,haplotype name,exon number,splice site type,protospacer,guide seq with PAM,PAM,enzyme class,CBE/ABE position,guide score,cDNA distuption pct
0,A*24:02:01:01,1,donor,GACCCCGCACTCACCTGCCC,GACCCCGCACTCACCTGCCCAGG,NGG,CBE,14,0.115790,0.067395
1,A*24:02:01:01,1,donor,CGCACTCACCTGCCCAGGTC,CGCACTCACCTGCCCAGGTCTGG,NGG,CBE,9,0.340876,0.067395
2,A*24:02:01:01,1,donor,GCACTCACCTGCCCAGGTCT,GCACTCACCTGCCCAGGTCTGGG,NGG,CBE,8,0.473178,0.067395
3,A*24:02:01:01,1,donor,CACCTGCCCAGGTCTGGGTC,CACCTGCCCAGGTCTGGGTCAGG,NGG,CBE,3,0.342684,0.067395
4,A*24:02:01:01,1,donor,ACCTGCCCAGGTCTGGGTCA,ACCTGCCCAGGTCTGGGTCAGGG,NGG,CBE,2,0.262362,0.067395
5,A*24:02:01:01,1,donor,GACCCCGCACTCACCTGCCC,GACCCCGCACTCACCTGCCCAGG,NGG,ABE,13,0.020324,0.067395
6,A*24:02:01:01,1,donor,CGCACTCACCTGCCCAGGTC,CGCACTCACCTGCCCAGGTCTGG,NGG,ABE,8,0.269631,0.067395
7,A*24:02:01:01,1,donor,GCACTCACCTGCCCAGGTCT,GCACTCACCTGCCCAGGTCTGGG,NGG,ABE,7,0.417329,0.067395
8,A*24:02:01:01,1,donor,CACCTGCCCAGGTCTGGGTC,CACCTGCCCAGGTCTGGGTCAGG,NGG,ABE,2,0.061594,0.067395
9,A*24:02:01:01,1,donor,ACCTGCCCAGGTCTGGGTCA,ACCTGCCCAGGTCTGGGTCAGGG,NGG,ABE,1,0.027279,0.067395


In [17]:
A_24_02_info_frm_url = parse_json(file_url="https://www.ebi.ac.uk/cgi-bin/ipd/api/allele/HLA00002")
A_24_02_info_frm_url

{'haplotype_name': 'A*01:02:01:01',
 'prevalence': ('C', 'C'),
 'genomic_features': [{'length': 300, 'start': 1, 'type': '5utr'},
  {'length': 73,
   'number': '1',
   'partial': False,
   'start': 301,
   'type': 'exon'},
  {'length': 130,
   'number': '1',
   'partial': False,
   'start': 374,
   'type': 'intron'},
  {'length': 270,
   'number': '2',
   'partial': False,
   'start': 504,
   'type': 'exon'},
  {'length': 241,
   'number': '2',
   'partial': False,
   'start': 774,
   'type': 'intron'},
  {'length': 276,
   'number': '3',
   'partial': False,
   'start': 1015,
   'type': 'exon'},
  {'length': 579,
   'number': '3',
   'partial': False,
   'start': 1291,
   'type': 'intron'},
  {'length': 276,
   'number': '4',
   'partial': False,
   'start': 1870,
   'type': 'exon'},
  {'length': 102,
   'number': '4',
   'partial': False,
   'start': 2146,
   'type': 'intron'},
  {'length': 117,
   'number': '5',
   'partial': False,
   'start': 2248,
   'type': 'exon'},
  {'length':